<a href="https://colab.research.google.com/github/davidharrisnet/genai/blob/main/llama2_13b_chat_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb

In [2]:
import sys
import os

In [3]:
!pip3 install virtualenv
!virtualenv llama_env

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 29.0 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 380ms
  creator CPython3Posix(dest=/content/llama_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==69.0.2, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [4]:
os.path.exists("/content/llama_env/bin")

True

In [5]:
sys.path.insert(0,"/content/llama_env/lib/python3.10/site-packages")

In [6]:
os.environ["PATH"] = "/content/llama_env/bin:" + os.environ["PATH"]

In [7]:
os.environ["PATH"]

'/content/llama_env/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin'

In [8]:
!which pip

/content/llama_env/bin/pip


In [9]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:0

In [15]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
import os

In [16]:

from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import os

In [17]:
from google.colab import drive
drive.mount('/content/drive/')




Mounted at /content/drive/


In [18]:
cuda.is_available()

True

In [19]:
model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-13b-chat-hf" , "models--meta-llama--Llama-2-13b-chat-hf", "snapshots", "1")
os.path.exists(model_id)

True

In [20]:

device = f'cuda:{cuda.current_device()}'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [21]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Prepare model, tokenizer: 298.241 sec.


In [22]:
print(model.get_memory_footprint())

7083970560


In [23]:
questions = [ "Write me a detailed scenario for a potential Russian invasion of Moldova.",

"Now identify one person in this scenario who would be a potential HUMINT source for the US Intelligence Community and write a short source dossier. Be sure to include their placement and access.",
"Write me three short intelligence information reports that came from that source about various events in the scenario.",
"Add dates and times for the intelligence reports." ]




In [24]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [25]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
from langchain import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.memory import  ConversationSummaryMemory


In [26]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=10000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

In [27]:
prompt_template = """<s>[INST] <<SYS>>
{{ You are an intelligence analyst}}<<SYS>>
###

Previous Conversation:
'''
{history}
'''

{{{input}}}[/INST]

"""
prompt  = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])

In [28]:
#memory = ConversationSummaryMemory(llm=llm)
memory = ConversationBufferWindowMemory(k=10)

chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [29]:
chain.run(questions[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"As an intelligence analyst, I have been tasked with preparing a detailed scenario for a potential Russian invasion of Moldova. Here is my assessment:\n\nScenario: Russian Invasion of Moldova\n\nBackground:\n\n* Moldova is a small, landlocked country located between Romania and Ukraine, with a population of approximately 3.5 million people.\n* The country has a diverse ethnic makeup, with ethnic Romanians, Ukrainians, and Russians being the largest groups.\n* Moldova declared independence from the Soviet Union in 1991 and has been a sovereign state since then.\n* The country has a relatively weak economy and is heavily dependent on foreign aid and remittances from Moldovan diaspora.\n\nReasons for Invasion:\n\n* Russia has long considered Moldova to be within its sphere of influence, and has been seeking to maintain control over the country and its resources.\n* Russia has been opposed to Moldova's membership in the European Union and NATO, viewing these organizations as a threat to it

In [30]:
chain.run(questions[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Person: Victor Kuznetsov, a high-ranking officer in the Moldovan Defense Ministry with access to classified information about the Moldovan military's capabilities and plans.\n\nSource Dossier:\n\nName: Victor Kuznetsov\nAge: 45\nNationality: Moldovan\nOccupation: High-ranking officer in the Moldovan Defense Ministry\nPlacement: Chisinau, Moldova\nAccess: Classified information about the Moldovan military's capabilities and plans\n\nBackground: Victor Kuznetsov is a career military officer and a graduate of the Russian Military Academy. He has served in various positions within the Moldovan Defense Ministry for the past 10 years, rising to the rank of colonel. Kuznetsov is well-connected within the Moldovan government and has access to classified information about the country's military capabilities and strategic plans. He is known to be a pro-Western and pro-democracy advocate, and has been critical of the Russian government's increasing influence in Moldova.\n\nMotivation: Kuznetsov 

In [31]:
chain.run(questions[2])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Report 1:\n\nTitle: Increased Russian Military Activity near the Moldovan Border\n\nDate: March 25, 2023\n\nSource: Victor Kuznetsov, High-ranking Officer in the Moldovan Defense Ministry\n\nSummary: Satellite imagery and drone surveillance have detected increased military activity in Russian-controlled territories near the Moldovan border. The information indicates that Russian troops and military equipment are being mobilized and deployed to the region. The intelligence suggests that the Russian government may be preparing for a potential invasion of Moldova.\n\nReport 2:\n\nTitle: Pro-Russian Protests in Chisinau\n\nDate: March 27, 2023\n\nSource: Victor Kuznetsov, High-ranking Officer in the Moldovan Defense Ministry\n\nSummary: Open-source intelligence suggests that pro-Russian protests have broken out in the capital city of Chisinau, Moldova. The protests are reportedly being led by pro-Russian groups and are calling for the Moldovan government to be overthrown and for Moldova t

In [32]:
chain.run(questions[3])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Report 1: Increased Russian Military Activity near the Moldovan Border\n\nDate: March 25, 2023, 16:30\n\nSource: Victor Kuznetsov, High-ranking Officer in the Moldovan Defense Ministry\n\nSummary: Satellite imagery and drone surveillance have detected increased military activity in Russian-controlled territories near the Moldovan border. The information indicates that Russian troops and military equipment are being mobilized and deployed to the region. The intelligence suggests that the Russian government may be preparing for a potential invasion of Moldova.\n\nReport 2: Pro-Russian Protests in Chisinau\n\nDate: March 27, 2023, 10:00\n\nSource: Victor Kuznetsov, High-ranking Officer in the Moldovan Defense Ministry\n\nSummary: Open-source intelligence suggests that pro-Russian protests have broken out in the capital city of Chisinau, Moldova. The protests are reportedly being led by pro-Russian groups and are calling for the Moldovan government to be overthrown and for Moldova to be a